Notebook is the exploration of the Vision Model from Deep SeekAI, A Chinese company lok

In [1]:
import torch
from transformers import AutoModelForCausalLM
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images

In [2]:
model_path = "deepseek-ai/deepseek-vl-7b-chat"
vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

preprocessor_config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


processor_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

In [ ]:
vl_gpt = AutoModelForCausalLM.from_pretrained(model_path,
                                              trust_remote_code=True,
                                              resume_download=True)

config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/81.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

### Error issue
Faced the error: ImportError: cannot import name 'Mapping' from 'collections' 
Had to modify the import Mapping to be collections.abc module. This had to be done in multiple files

import collections.abc
for type_name in collections.abc.__all__:
    setattr(collections, type_name, getattr(collections.abc, type_name))

    
- aimachine/lib/python3.10/site-packages/attrdict/default.py

- aimachine/lib/python3.10/site-packages/attrdict/merge.py

- aimachine/lib/python3.10/site-packages/attrdict/mixins.py

- aimachine/lib/python3.10/site-packages/attrdict/mapping.py

In [ ]:
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

In [ ]:
conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>Describe each stage of this image.",
        "images": ["./images/training_pipelines.png"]
    },
    {
        "role": "Assistant",
        "content": ""
    }
]

In [ ]:
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversation=conversation,
    images=pil_images,
    force_batchify=True
).to(vl_gpt.device)

In [ ]:
input_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

In [ ]:
outputs = vl_gpt.language_model.generate(
    input_embeds=input_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    user_cache=True
)